# ORGANISM: Homo_sapiens
# Datasource: https://downloads.thebiogrid.org/BioGRID/Release-Archive/BIOGRID-3.5.182/

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df_original = pd.read_csv("data/BIOGRID-ORGANISM-Homo_sapiens-3.5.182.tab2.txt", sep='\t')

/home/ranap/anaconda3/envs/MLenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_original.head()

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,...,9006895,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,...,11309420,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
2,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,...,8599089,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
3,278,2624,5371,108894,111384,-,-,GATA2,PML,DCML|IMD21|MONOMAC|NFE1B,...,10938104,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
4,418,6118,6774,112038,112651,RP4-547C9.3,-,RPA2,STAT3,REPA2|RP-A p32|RP-A p34|RPA32,...,10875894,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID


In [4]:
df = df_original[["Official Symbol Interactor A", "Official Symbol Interactor B"]]

In [5]:
df.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,MAP2K4,FLNC
1,MYPN,ACTN2
2,ACVR1,FNTA
3,GATA2,PML
4,RPA2,STAT3


In [6]:
G = nx.from_pandas_edgelist(df, 'Official Symbol Interactor A', 'Official Symbol Interactor B', create_using=nx.DiGraph())

In [7]:
print(G.is_directed(), len(G.nodes), len(G.edges()))

True 23649 428893


In [8]:
# deleting self-loop edges
for n, nbrs in G.adj.items():
    if n in nbrs:
        G.remove_edge(n,n)

In [9]:
print(G.is_directed(), len(G.nodes), len(G.edges()))

True 23649 426026


### 3 nodes chain excluding common regulator 

In [10]:
def common_regulator(G, node1, node2):
    in_node1 = [a[0] for a in G.in_edges(node1)]
    in_node2 = [b[0] for b in G.in_edges(node2)]
    for i in in_node1:
        if i in in_node2:
            return True
        return False

In [11]:
common_regulator(G,'marA','putP')

In [12]:
node_set = set() # to save the node list
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if(node3 not in [node1, node2] and (node3 not in G.neighbors(node1)) and not(common_regulator(G, node1, node3))): 
                    print(node1, node2, node3)
                    node_set.add(node1); node_set.add(node2); node_set.add(node3);

MAP2K4 Flna FURIN
ACTN2 GOLGA7 GOLGA3
ACVR1 BAMBI SMAD7
ACVR1 TGFBRAP1 SMAD4
ACVR1 ARV1 CREB1
ACVR1 CD320 PRDX2
GATA2 PRR20A SELV
PML C1QTNF6 SGTA
PML CACUL1 AR
RPA2 SMARCAL1 SULT1A1
RPA2 RNASEH1 TM9SF1
RPA2 ESRP2 NUMBL
RPA2 RBM26 CRCP
RPA2 TMEM33 ABCC1
RPA2 ARG1 ARG2
RPA2 MYO1F OSTF1
STAT3 ANKRD26P1 RBFOX3
GGA3 SYNRG SCAMP1
ARFIP2 LY6D PKIB
ARFIP1 PGM2L1 PITPNB
XRN1 C1GALT1 C1GALT1C1
XRN1 ATP5G2 TDP2
ALDOA SUMO4 NFKBIA
APP ACHE COL4A1
APP C1orf116 HOMER1
APP GLRB GPHN
APP GPR15 YWHAB
APP ZNF641 KDM1A
APP CTT1 UBE2I
APP ACADL SYT2
APP CES3 TMTC4
APP CKAP2 FZR1
APPBP2 MFSD3 PCDHGC3
APPBP2 SYNRG SCAMP1
APPBP2 ST8SIA5 SEPN1
APPBP2 C1orf50 EPM2AIP1
APPBP2 LONRF3 NOTCH2NL
APPBP2 DCD IL24
DAB1 PRR20A SELV
DAB1 TMTC3 COTL1
EP300 JMY TTC5
EP300 HSP90AA2P HSP90AB1
EP300 HIST2H2AB APP
EP300 FOSB SRA1
EP300 MSTO1 NR1H3
EP300 CRTC1 MAP3K1
EP300 CYP1B1 PGRMC1
ARRB2 STC2 PTN
ARRB2 EIF2AK4 GGNBP1
ARRB2 NOP10 NHP2
ARRB2 ACTG2 NEB
ARRB2 DCD IL24
ARRB2 HDGFRP2 HIST2H3C
ARRB2 BOLA2B PGK1
GRB2 SH3D19 SH3Y

PIK3R3 SERF1A APP
PIK3R3 BRK1 DTNBP1
PIK3R3 TNNC2 TNNI3
PIK3CA BEX1 TEX11
PIK3CA TSPAN14 ADAM10
APOA1 APOC1 HMP19
APOA1 CP LTF
APOA1 IGKC SMARCD1
APOA1 KRT16 TCHP
APOA1 DGAT1 GABRA4
PFN2 NAT6 APP
PRKAR1A PALM2-AKAP2 TFG
PRKAR1A SPHK2 HIST1H3A
AKAP1 AMY1A AKAP8
FER TMF1 AR
NEK2 SPTBN4 ANK3
NEK2 PRSS3 TFPI
NEK2 CDSN CEP55
HABP4 MRPL17 RPS23
HABP4 CCDC86 DLST
HABP4 GTPBP10 PCBP4
HABP4 MORC2 HDAC4
HABP4 SURF6 RRS1
HABP4 TBL2 MEOX2
HABP4 C7orf50 APP
AR DENND5A XRN2
AR MED12L NCAPD2
AR HOTAIR EZH2
PDPK1 TBL2 MEOX2
PRKCZ DENND5A XRN2
PRKCZ MRPL54 MRPS7
RHOG ELMOD2 HOXC4
RAB5A ATE1 SUV39H1
RAB5A YTHDF3 YTHDF1
EXOC8 TLN2 TWF2
RALA GLS2 GRHPR
RALA GPT CFL1
BRCA2 SPHK2 HIST1H3A
BRCA2 DDX3Y CCNB2
BRCA2 DNAH9 FAM103A1
BRCA2 PPP1R17 APP
BRCA2 KIAA1468 TTC4
BRCA2 C4orf22 APP
BRCA2 CLHC1 CEP170P1
BRCA2 CCDC24 REN
RAD51AP1 IGHG4 TF
HBP1 TMEM37 UBQLN1
HBP1 NOL6 RPS4X
HBP1 C7orf50 APP
TRAP1 EIF2AK4 GGNBP1
AATF NOL6 RPS4X
THOC1 MRPS17 KIFAP3
RBL2 GRAMD4 APP
LMO2 CLHC1 CEP170P1
LMO2 CCDC36 ZGPAT
LMO2 CCZ1 

HNRNPL KLK4 KCNIP4
HNRNPL MICAL2 FLT1
HNRNPL PHF14 PCBD2
HNRNPL KIAA0232 SLC35F6
HNRNPL PTDSS1 STOM
HNRNPL FAM13A UNC119
HNRNPL MBNL2 ACOX1
HNRNPL RAMP1 CALCRL
HNRNPL CASC3 GRB2
HNRNPL ENDOD1 STOM
HNRNPL ZNF609 EXOC5
HNRNPL NUP210 CCRN4L
HNRNPL SEL1L3 FAM63A
HNRNPL ATP11A TMEM30A
HNRNPL DENND5A XRN2
HNRNPL ZCCHC11 TIFA
HNRNPL ARHGEF18 GNB1
HNRNPL SLC35A3 TP63
HNRNPL PLD3 HNRNPH3
HNRNPL MYRIP C1orf216
HNRNPL CKAP2 FZR1
HNRNPL CPAMD8 SCTR
HNRNPL ANKMY1 RBPMS
HNRNPL AIG1 RCHY1
HNRNPL GULP1 SMC2
HNRNPL CXXC5 VDR
HNRNPL ZNF44 RDH13
HNRNPL STAG3L1 AGTRAP
HNRNPL ANKIB1 KPNA2
HNRNPL EPB41L4B LAMC3
HNRNPL EXD3 DAZAP2
HNRNPL UCKL1 BCS1L
HNRNPL SOBP SUMO2
HNRNPL NADSYN1 MYO15B
HNRNPL FGGY EAF1
HNRNPL PLCXD1 NMT1
HNRNPL G2E3 SPECC1L
HNRNPL WDR11 NAA40
HNRNPL ASH1L HIST1H3A
HNRNPL KCNQ5 KCNQ3
HNRNPL ESYT2 SLC25A17
HNRNPL CYP4F11 POT1
HNRNPL FAM111A TP53
HNRNPL TSPYL2 PSMD3
HNRNPL WNK2 SGK1
HNRNPL ERI3 TTK
HNRNPL SLC35F5 SHOC2
HNRNPL TTYH3 NEDD4
HNRNPL TSPAN10 ADAM10
HNRNPL RHBDD1 VCP
HNRNPL ABLIM2 

CDK14 IGHA2 MUC5AC
TLK1 KIAA1468 TTC4
ASF1A hist1h3g BANF1
ASF1A ANKLE2 PPP2R1A
SMAD2 Arhgef7 PAK2
SMAD2 Lmo2 RBBP8
SMAD2 DDX3Y CCNB2
SMAD2 C1orf116 HOMER1
SMAD2 DGKA ORC5
SMAD2 CYP11A1 FDX1
SMAD2 ST5 ABL1
SMAD2 PAPPA PRG2
Ppp2r2d FAM122B CBY1
Chek1 HDGFRP2 HIST2H3C
Csnk1d RASIP1 HRAS
Csnk1d SLC35E1 IFT52
Csnk1d CD109 EGFR
Fancl TMEM33 ABCC1
Tgfbr1 ACAD8 ACADSB
Tgfbr1 SLC35E1 IFT52
Ppp6c DLGAP4 BIN1
BMPR1B BAMBI SMAD7
Ppp2ca TFB2M SLC25A29
Ppp2ca ZCCHC9 ZNF574
Ppp2ca ERI1 HNRNPD
SMURF1 Ecsit TRAF6
SMURF1 Arhgef7 PAK2
SMURF1 Dnajc7 HSP90AA1
SMURF1 Rapsn CHRNA9
SMURF1 Rnf138 CACNA1A
SMURF1 E2F7 HIF1A
SMURF1 ACAP2 RAB11FIP3
SMURF1 TAOK3 TRAF2
SMURF1 MAP3K9 HSP90AA1
SMURF1 ADRBK2 CXCR4
SMURF2 Rnf2 HIST2H2AC
SMURF2 MARS2 APP
RNF5 CYB561A3 AOC1
NCOR1 Nr3c1 HSP90AA1
ABHD16A SPAG7 UBA2
DNAJC1 SERPINA3 ZNRD1
DNAJC1 RMND1 FAM9B
GPRC5C GOLPH3L LNX1
CSNK2B DENND5A XRN2
CSNK2B CCDC115 APP
CSNK2B GAMT FARS2
CSNK2B SEC11A SLC35F6
CSNK2A2 HDGFRP2 HIST2H3C
CSNK2A2 LAC1 CCDC155
CSNK2A2 ZNF592 FAM115A
CS

SPRY2 VANGL1 RNF41
SPRY2 TMEM14B NCALD
ZC3H10 PRR20A SELV
SQSTM1 Grin2a CHN1
SQSTM1 INA RAB11FIP5
SQSTM1 GLG1 SELE
SQSTM1 GFM2 TRIM9
SQSTM1 CD48 LCK
TSNAX PDF MYO5B
AKAP17A MYH4 ACTN2
SEC23B UGT1A1 CYP3A4
NBR1 SYT4 SYNCRIP
PAFAH1B2 REXO1 TCEB3
PAFAH1B3 CHMP7 ARHGAP17
BEGAIN HID1 INPPL1
BEGAIN QSOX1 UBE2D2
ACTG1 BRK1 DTNBP1
PDE9A C9orf106 PSMG3
FAM50B MRPL54 MRPS7
KHDRBS2 TMEM14B NCALD
KHDRBS2 PTER ABCE1
USE1 TYW5 POTEF
CIAO1 WDR43 NAP1L1
CIAO1 UCKL1 BCS1L
CIAO1 CNP SIRT3
PAICS TMEM177 TSR1
CEP76 ABR APP
CEP76 PPFIBP1 KANK2
CEP76 EP400NL APP
PTPN9 FSIP1 APP
PTPN9 TUBA3D TUBB4B
UPRT UCKL1 BCS1L
MTUS2 ZNF415 KRTAP4-12
MTUS2 GTPBP10 PCBP4
MTUS2 SNHG11 KRTAP10-3
MTUS2 ALS2CR11 BRCA1
MTUS2 LOC153684 KRTAP10-8
MTUS2 ZNF844 KRTAP10-9
MTUS2 LINC00636 KRTAP10-3
MTUS2 ATP11C TMEM30A
MTUS2 MYADM ARF6
MTUS2 CD109 EGFR
SOX30 BAMBI SMAD7
DYNC1I2 MLLT11 S100B
DYNC1I2 TFB2M SLC25A29
DYNC1I2 TGM3 PPME1
DYNC1I2 ATE1 SUV39H1
DYNC1I2 KRT9 KRT1
EMD ACTG2 NEB
FATE1 SYNPR KIAA1715
FATE1 TMEM74 ATG16L1
FATE1 S

...

IPPK ARV1 CREB1
IPPK TTYH3 NEDD4
CD1A DGAT1 GABRA4
MFF CCDC126 HTT
RABGGTA DPCD APP
RCCD1 ZNF609 EXOC5
FDXACB1 RBMXL2 RABGGTA
TRNT1 NARS2 DARS2
PPAT CCZ1 RAB7A
PPAT NPEPL1 HMBS
Nxf1 NUPL2 vpr
Nxf2 NUPL2 vpr
MTNR1B ITM2C MVB12B
MTNR1B GPM6B NME2P1
MTNR1B TMEM14A LXN
MTNR1B DMXL2 WDR7
MTNR1B DYNC2H1 DYNC2LI1
MTNR1B COX11 RPL19
MTNR1B CDH11 CDH2
MTNR1B CDH4 CDH2
MTNR1B FSTL1 APPBP2
MTNR1B APOL6 KRTAP10-8
MTNR1B SLC12A4 TMEM43
MTNR1B SLC4A7 CFTR
ZNF3 SHCBP1 ZUFSP
TMEM248 ABCA2 CDK5RAP2
TPTE2 TBC1D31 ZNHIT6
TPTE2 RMND1 FAM9B
TPTE2 KIAA0232 SLC35F6
CHST10 SLC22A18 RNF167
VKORC1 SPTSSA MBOAT7
VKORC1 HPR ORF57
VKORC1 DPM2 PIGP
VKORC1 TMEM45B UBE2G2
VKORC1 PIGH PIGA
VKORC1 APOC1 HMP19
VKORC1 RMDN1 SHC1
VAMP5 ESYT2 SLC25A17
VAMP5 GPX8 GAPDHS
VAMP5 TMX3 APP
TM2D3 C17orf80 KCNA5
TM2D3 FSTL1 APPBP2
COMTD1 MSTO1 NR1H3
ZNF324B ZNF624 MTUS2
ZNF324B NOP14 PAXIP1
ZNF324B PTCD1 SDHC
ZNF324B MRPL17 RPS23
ZNF324B TSPYL2 PSMD3
ZNF688 CCDC36 ZGPAT
B4GAT1 PPM1L MAP3K7
B4GAT1 PTPLB BCAP31
B4GAT1 SLC47A1 TPM2
F

SPPL2B FAM20B ASPHD2
SPPL2B CMTM4 MRPL2
MAGEA10 QSER1 PLEC
MAGEA10 ERI1 HNRNPD
MAGEA10 MTCL1 MARK2
FRMD8P1 FRMD8 APP
POPDC2 DHRS7 ETV5
POPDC2 ATP11C TMEM30A
POPDC2 AAAS REL
VSX1 USP9Y UBC
KNOP1 ZNF106 CREBBP
PLAC1 EPC2 EZH2
LYG2 NID2 KRTAP10-3
FAM177A1 SAR1B HNRNPH1
PSG11 GAPDHS APP
SEC11C SPCS3 VDAC3
HLA-DQB2 ATE1 SUV39H1
HS2ST1 PIGU STOM
HS2ST1 USP9Y UBC
CCDC183 TMF1 AR
CCDC183 INA RAB11FIP5
MYO19 CPM GTPBP3
MYO19 SIPA1 BRD4
MYO19 ST5 ABL1
MYO19 GPRC5A GTF2F2
MYO19 MYADM ARF6
TMEM206 FAM114A2 APP
TMEM206 TTYH3 NEDD4
SCGB1D1 CNTNAP3 APBA1
SCGB1D1 SEL1L3 FAM63A
SCGB1D1 CD109 EGFR
SCGB1D1 SEMA4G RBPMS
SCGB1D1 CERCAM GOPC
SCGB1D1 MRC2 VDAC3
CBWD1 SERAC1 APP
TMEM63A ATP11C TMEM30A
TMEM63A ZDHHC9 CREB3
SLC25A41 NARS2 DARS2
CLDND1 GOLGA7 GOLGA3
CLDND1 CYTB MVP
CST4 MUC5AC ITGB4
METTL6 TYW5 POTEF
C20orf96 TYW5 POTEF
ALOXE3 ZFHX4 CHD4
INTU CNP SIRT3
SERPINC1 BTD MYO1D
SPIN2B HIST1H2AG MTRR
SPIN2B HIST2H2AB APP
SPIN2B FAM122B CBY1
HEATR6 GAS8 CIAO1
FAM153B EIF2AK4 GGNBP1
FAM153B TYW5 POTEF
FAM

CEP104 CKAP2 FZR1
CEP104 DLG5 CTNNB1
LYPLAL1 MSRB2 NEDD8
TMEM17 ANKLE2 PPP2R1A
TMEM17 CCDC115 APP
TMEM17 CCDC47 TOM1L1
TMEM17 DHRS7 ETV5
TMEM17 EPN2 RALBP1
TMEM17 ESYT2 SLC25A17
TMEM17 FNDC3A APP
TMEM17 FZD6 SFRP1
TMEM17 GPR180 CLU
TMEM17 ITM2C MVB12B
TMEM17 JAG2 MIB2
TMEM17 OSBPL11 APP
TMEM17 PANX1 GNAS
TMEM17 PLD3 HNRNPH3
TMEM17 PPFIBP1 KANK2
TMEM17 RAB23 POTEKP
TMEM17 RHBDD2 IFT27
TMEM17 SCYL3 LHX8
TMEM17 SEC11A SLC35F6
TMEM17 SLC12A4 TMEM43
TMEM17 SLC12A6 VAV2
TMEM17 SLC38A1 FATE1
TMEM17 SLC38A2 APP
TMEM17 SLC4A7 CFTR
TMEM17 SLC6A9 STX1A
TMEM17 SPCS3 VDAC3
TMEM17 TMEM115 SYNE4
TMEM17 TMEM87A AGPS
TMEM17 VANGL1 RNF41
TMEM17 WDR11 NAA40
TMEM17 GPRC5A GTF2F2
TMEM17 TENM1 SORBS1
TMEM17 LPHN2 NCK1
KCNC4 RHBDD1 VCP
CELA3A CNTNAP3 APBA1
PNLIPRP1 ITGA8 NPNT
AOC1 NT5M PIK3CD
PTPDC1 MAPK1IP1L SUMO1P1
NDUFA11 UQCR10 RAB11A
NDUFA11 TMEM14C CCDC155
TIGD6 DYNC2H1 DYNC2LI1
TIGD6 HDGFRP2 HIST2H3C
MBOAT1 SLC35E1 IFT52
MBOAT1 PTDSS2 STOM
PCDHB15 HIGD1A IPO13
PCDHB15 TBCK APP
ZNF526 DEPDC7 GTF2B
TYW3

FADS3 FNDC3A APP
MANSC1 ANKRD13C PTGDR
MANSC1 SLC35F5 SHOC2
MANSC1 DIRAS1 APP
MANSC1 CMTM4 MRPL2
MANSC1 PLEKHB1 SH2D2A
MANSC1 LRP10 GGA3
MANSC1 VANGL1 RNF41
MANSC1 RHBDD1 VCP
CLEC3A RNASEH1 TM9SF1
CLEC3A ZNF593 LNX1
CLEC3A RBM26 CRCP
PCDH12 CELSR1 PSAP
HTR3A OCA2 EVI5L
HTR3A HLA-DOA REL
HTR3A UGT3A2 ACAT1
HTR3A LRRC8D APP
HTR3A PANX1 GNAS
HTR3A LPPR3 BAG6
HTR3A ABCA2 CDK5RAP2
HTR3A GRID1 HBB
HTR3A EXTL3 XIAP
HTR3A SLC47A1 TPM2
HTR3A TMTC3 COTL1
HTR3A C6orf89 PTRHD1
HTR3A C1GALT1C1 LGALS3
HTR3A TMX4 TOMM22
HTR3A GPX8 GAPDHS
ACOT12 ACOT11 THUMPD3
ZNF669 SESTD1 GSN
SCGB1D4 CNTNAP3 APBA1
SCGB1D4 ITGA8 NPNT
SCGB1D4 CERCAM GOPC
FZD10 MYADM ARF6
C19orf45 CADPS2 BZRAP1
C19orf45 TARBP1 ASNA1
TSPYL6 TSPYL2 PSMD3
TSPYL6 DMXL2 WDR7
TSPYL6 TRAF7 MAP3K3
TSPYL6 POLG TTC3
TSPYL6 KIAA0232 SLC35F6
CACNG5 ARHGEF39 TIFA
CACNG5 TTLL5 CREB3
AMZ1 BRK1 DTNBP1
FAM19A3 MRC2 VDAC3
FAM19A3 TARBP1 ASNA1
FAM19A3 PPP2R3B TMEM14B
SLC39A5 C3orf33 APP
SLC39A5 GOLPH3L LNX1
SLC39A5 LRP10 GGA3
SLC39A5 SLC22A18 RNF167
SLC3

MIR106A PDCD11 SUPT6H
MIR106B DDX3Y CCNB2
MIR106B PDCD11 SUPT6H
MIR106B NOL6 RPS4X
MIR107 DDX3Y CCNB2
MIR107 PDCD11 SUPT6H
MIR107 NOL6 RPS4X
MIR122 PDCD11 SUPT6H
MIR122 NOL6 RPS4X
MIR128-1 RBM12B SF3A1
MIR128-1 APOBEC3B MSL2
MIR128-1 PDCD11 SUPT6H
MIR128-1 NOL6 RPS4X
MIR128-1 LSM10 ZNF473
MIR128-1 CASC3 GRB2
MIR128-2 APOBEC3B MSL2
MIR128-2 DDX3Y CCNB2
MIR128-2 PDCD11 SUPT6H
MIR128-2 NOL6 RPS4X
MIR138-1 PDCD11 SUPT6H
MIR138-2 APOBEC3B MSL2
MIR138-2 PDCD11 SUPT6H
MIR138-2 NOL6 RPS4X
MIR138-2 ZNF106 CREBBP
MIR140 APOBEC3B MSL2
MIR140 PDCD11 SUPT6H
MIR140 NOL6 RPS4X
MIR141 APOBEC3B MSL2
MIR141 PDCD11 SUPT6H
MIR141 LARP6 APP
MIR143 ZCCHC11 TIFA
MIR143 APOBEC3B MSL2
MIR143 DDX3Y CCNB2
MIR143 PDCD11 SUPT6H
MIR143 NOL6 RPS4X
MIR145 APOBEC3B MSL2
MIR145 PDCD11 SUPT6H
MIR145 LARP6 APP
MIR155 APOBEC3B MSL2
MIR155 PDCD11 SUPT6H
MIR155 NOL6 RPS4X
MIR199A1 PDCD11 SUPT6H
MIR199A1 NOL6 RPS4X
MIR199A2 APOBEC3B MSL2
MIR199A2 PDCD11 SUPT6H
MIR199A2 NOL6 RPS4X
MIR200A APOBEC3B MSL2
MIR200A PDCD11 SUPT6H
M

In [13]:
# H = G.subgraph(list(node_set))
# pos = nx.circular_layout(H)
# nx.draw(H, pos, cmap = plt.get_cmap('jet'), node_size = 2000)
# nx.draw_networkx_labels(H, pos)
# plt.show()

### 4 nodes chain

In [14]:
# pattern 2 search
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if (G.out_degree(node3)>1 or G.out_degree(node3)>1 or node3 in [node1, node2]):
                    continue
                else:
                    for node4 in G.neighbors(node3):
                        if(node4 not in [node1, node2, node3] and (node4 not in G.neighbors(node1)) and not(common_regulator(G, node1, node4))):
                            print(node1, node2, node3, node4)

XRN1 C1GALT1 C1GALT1C1 LGALS3
GRB2 KRT82 KRT16 TCHP
CTNNB1 CDH6 CDH18 MYOG
CTNNB1 CDH7 FGF21 MAPK7
FLNA CPM GTPBP3 PTAFR
GNAI2 KIAA1919 DYNC2H1 DYNC2LI1
HSPG2 FBLN2 FBN1 MFAP2
MAD2L2 GPX8 GAPDHS APP
HTT MIPEP INA RAB11FIP5
YWHAQ RNASE2 PCP2 GNAI3
TCF3 Tcf12 ASCL3 APP
TCF3 Tcf3 ASCL3 APP
LMNA PLLP MS4A12 FAM3A
NFKB1 GPX8 GAPDHS APP
SNCAIP RGP1 NBPF22P PARVG
NRAS SCYL3 LHX8 SUV39H1
CD27 C1GALT1 C1GALT1C1 LGALS3
NTRK1 THYN1 SHPK TRAF3
HRAS PLLP MS4A12 FAM3A
HRAS OLR1 F5 PROS1
KRAS RGP1 NBPF22P PARVG
KRAS PLLP MS4A12 FAM3A
KRAS OLR1 F5 PROS1
TP53 HYAL4 GAPDHS APP
DLG3 KRT82 KRT16 TCHP
XPO1 SCYL3 LHX8 SUV39H1
CDC6 PPP2R3B TMEM14B NCALD
RNF4 GPX8 GAPDHS APP
COIL CYB5R2 FFAR2 IGF1R
CSTF2 PAPOLB TTC38 PFAS
HNRNPL C14orf79 FAT3 HPRT1
DCTN1 MIPEP INA RAB11FIP5
ESR2 SCYL3 LHX8 SUV39H1
TRAF1 CYB5R2 FFAR2 IGF1R
TRAF2 CYB5R2 FFAR2 IGF1R
ILK MIPEP INA RAB11FIP5
GOLGA2 CYB5R2 FFAR2 IGF1R
GOLGA2 C12orf50 GAPDHS APP
GOLGA2 THYN1 SHPK TRAF3
PRKAG1 KIAA1919 DYNC2H1 DYNC2LI1
E2F6 CST4 MUC5AC ITGB4
PABPC1 P

### 5 nodes chain

In [15]:
# pattern 2 search
for node1 in G.nodes:
    for node2 in G.neighbors(node1):
        if (G.out_degree(node2)>1 or G.out_degree(node2)>1 or node2==node1):
            continue
        else:
            for node3 in G.neighbors(node2):
                if (G.out_degree(node3)>1 or G.out_degree(node3)>1 or node3 in [node1, node2]):
                    continue
                else:
                    for node4 in G.neighbors(node3):
                        if (G.out_degree(node4)>1 or G.out_degree(node4)>1 or node4 in [node1, node2, node3]):
                            continue
                        else:
                            for node5 in G.neighbors(node4):
                                if(node4 not in [node1, node2, node3, node4] and (node5 not in G.neighbors(node1)) and not(common_regulator(G, node1, node5))):
                                    print(node1, node2, node3, node4, node5)                            

Comment: No chain of length 5 found